In [20]:
#Imports
#Loading our data
import csv as csv
import cv2
import numpy as np

#Keras imports
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, MaxPooling2D, Activation
from keras import backend as K

#Matplotlib
import matplotlib.pyplot as plt

### Loading the data

In order to load our data we firstly read the driving_log.csv file which contains the paths to the left, center and right cameras and also info about steering angle, throttle, break and speed. 

We will want to automate the loading process in order to easily load our own data which is stored in different directories. For that we used several libraries such as csv (read csv files) and cv2 (read images).

In [23]:
#Reading the files

#Reading the csv file
def readCSV(in_path, in_dir_path):
    lines = []
    with open(in_path) as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None) #Skip header
        for line in reader:
            #Correct images paths
            line[0] = in_dir_path + line[0].split('/')[-1]
            line[1] = in_dir_path + line[1].split('/')[-1]
            line[2] = in_dir_path + line[2].split('/')[-1]
            
            lines.append(line)
            
    return lines

def readData_Basic(in_lines):
    images = []
    measurements = []
    
    for line in in_lines:
        image = cv2.imread(line[0])
        images.append(image)
        
        measurement = float(line[3])
        measurements.append(measurement)
    
    return np.array(images), np.array(measurements)

In [24]:
imgs_path = './data/IMG/'
csv_path = './data/driving_log.csv'

csv_lines = readCSV(csv_path, imgs_path)

X_train, y_train = readData_Basic(csv_lines)

print(type(y_train))

print('Data loaded successfully!')

<class 'numpy.ndarray'>
Data loaded successfully!


In [4]:
print(type(y_train[0]))

<class 'numpy.float64'>


### Data preprocessing

To grayscale?

In [22]:
def preprocess_image(in_image):
    return cv2.cvtColor(in_image, cv2.COLOR_BGR2YUV)

### Basic neural network using Keras

We will implement a basic neural network in order to verify that everything is working before implementing a more complex model. This network will just going to be a flattened image connected to a single output node. This single output node will predict the stearing angle, thus converting this model into a regression network. In contrast with a classification network, we may apply a softmax activation function to the output layer. Nevertheless in this case we will not use an activation function. We will directly predict the steering measurement. 

For this basic implementation we will use Keras as a library which works with tensorflow as backend. This will simplify our implementation and will be great for prototyping. Let's go ahead!

Improvement 1: In order to improve our model we need to preprocess our input data. For that we will add two preprocessing steps: normalization and  mean centering the data. We will add a lambda layer to our model. After doing this, we can decrease the training epochs a lot. We will fix the number of epochs in 2.

In [7]:
#Model definition

model = Sequential()
#Lambda layer for normalizing our data. In order to mean center the data, we will
#need to substract -0.5 (shifting the model down) to the normalized data. 
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Flatten(input_shape=(160,320,3)))
model.add(Dense(1))

#Model compilation
#For the loss function we will use Mean Squared Error (MSE). We will minimize the 
#error between the steering measurement which the network predicts and the ground 
#truth steering measurements provided by the dataset
model.compile(loss='mse', optimizer='adam')
#we also shuffle the data and split off 20% of the data to use for a validation set
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs= 2)

#Keras by default will run 10 epochs. Nevertheless with 10 epochs we will 
#overfit the training data. For that reason we will only perform 6 epochs
model.save('basic_model.h5')

K.clear_session()

Train on 6428 samples, validate on 1608 samples
Epoch 1/2
6428/6428 [==============================] - 2s 384us/step - loss: 1.8318 - val_loss: 0.7415
Epoch 2/2
6428/6428 [==============================] - 2s 370us/step - loss: 2.9713 - val_loss: 2.7356


#### We will try to autonomous drive the car with this model

### Trying a more complex network such as LeNet-5 architecture

We will implement LeNet-5 architecture using Keras.

In [17]:
model1 = Sequential()
#Lambda layer for normalizing our data. In order to mean center the data, we will
#need to substract -0.5 (shifting the model down) to the normalized data. 
model1.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=X_train.shape[1:]))

#First set of CONV => RELU => POOL
model1.add(Conv2D(3, (5, 5), input_shape= X_train.shape[1:]))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size= (2, 2), strides= (2, 2)))

#Second set of CONV => RELU => POOL
model1.add(Conv2D(6, (5, 5)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size= (2, 2), strides= (2, 2)))

#Setting the FCs layers
model1.add(Flatten())
model1.add(Dense(120))
model1.add(Dense(84))

#Output layer
model1.add(Dense(1))

#Model compilation
#For the loss function we will use Mean Squared Error (MSE). We will minimize the 
#error between the steering measurement which the network predicts and the ground 
#truth steering measurements provided by the dataset
model1.compile(loss='mse', optimizer='adam')
#we also shuffle the data and split off 20% of the data to use for a validation set
model1.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs= 2)

#Keras by default will run 10 epochs. Nevertheless with 10 epochs we will 
#overfit the training data. For that reason we will only perform 6 epochs
model1.save('lenet_model.h5')

K.clear_session()

Train on 6428 samples, validate on 1608 samples
Epoch 1/2
6428/6428 [==============================] - 4s 676us/step - loss: 1.5497 - val_loss: 0.0136
Epoch 2/2
6428/6428 [==============================] - 4s 629us/step - loss: 0.0113 - val_loss: 0.0120


### Data augmentation

In order to increase the number of data we have driven the car in the opposite direction along the routes. At the same time we can flip the images and also taking the opposite sign of steering measurements

In [21]:
def dataAugmentation(in_image, in_measurement):
    image_flipped = np.fliplr(image)
    measurement_flipped = -measurement
    
    return image_flipped, measurement_flipped

### Using Multiple Cameras

Up to this point we only used the center camera. But, using side cameras should be a great decision because we will have three times more data. And also, using these images we will teach the network how to steer back to the center if the vehicle starts drifting off to the side. 

The simulator captures images from three cameras mounted on the car: a center, right and left camera. That’s because of the issue of recovering from being off-center. In the simulator, you can weave all over the road and turn recording on and off to record recovery driving. In a real car, however, that’s not really possible. At least not legally.


In [25]:
def readData_Advanced(in_lines, correction=0.25):
    images = []
    measurements = []
    
    for line in in_lines:
        #Images
        center_img = cv2.imread(line[0])
        left_img = cv2.imread(line[1])
        right_img = cv2.imread(line[2])
        
        center_img = preprocess_image(center_img)
        left_img = preprocess_image(left_img)
        right_img = preprocess_image(right_img)
        
        images.append(center_img)
        images.append(left_img)
        images.append(right_img)
        
        #Measurements
        center_steer = float(line[3])
        left_steer = center_steer + correction
        right_steer = center_steer - correction

        measurements.append(center_steer)
        measurements.append(left_steer)
        measurements.append(right_steer)
    
    return np.array(images), np.array(measurements)

### nVidia Model



In [19]:
def NvidiaArchitecture():
    model = Sequential()
    #Preprocessing
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=X_train.shape[1:]))
    model.add(Cropping2D(cropping=((50,20), (0,0))))
    
    model.add(Conv2D(24,(5,5), subsample=(2,2), activation='relu'))
    model.add(Conv2D(36,(5,5), subsample=(2,2), activation='relu'))
    model.add(Conv2D(48,(5,5), subsample=(2,2), activation='relu'))
    model.add(Conv2D(64,(3,3), activation='relu'))
    model.add(Conv2D(64,(3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    
    return model
    